资本资产定价模型（英语：Capital Asset Pricing Model，缩写：CAPM）又称资本资产价格决定模型，为现代金融市场价格理论的支柱，广泛应用于投资决策和公司理财领域。于1960年代由美国学者威廉·夏普（William Sharpe）、林特尔（John Lintner）、崔诺（Jack Treynor）和莫辛（Jan Mossin）等人在现代投资组合理论的基础上发展。

CAPM主张投资组合的回报率只跟系统性风险有关。使用CAPM时，投资组合应已完全多角化，即包含所有风险性资产，如股票及债券，以消除个别证券的非系统性风险。

E(r(I)]=r_{f}+beta *[E(r(m))-r(f)],

E(r(m))是市场投资组合的期望回报率，通常用股票价格指数回报率的平均值或所有股票的平均回报率来代替

E(r(m))-r(f)是市场风险溢价（Risk Premium），即市场投资组合的期望回报率与无风险回报率之差


CAPM模型是建立在一系列假设的基础上的，其中主要包括：

1. 同素性预期假设，所有投资者拥有相同的预期，即对所有资产回报的均值、方差及协方差等，投资者均有完全相同的预期。
2. 所有投资者均可以无风险利率无限制地借入或贷出资金。
3. 所有投资者均追求单期财富的期望效用最大化，并以各备选组合的期望收益和标准差为基础进行组合选择。
4. 市场拥有充分的流动性且没有交易成本，所有资产均可被完全细分。
5. 所有投资者均为价格接受者，即任何一个投资者的买卖行为都不会对股票价格产生影响。
6. 资产数量之总和固定。

Step 1: import libraries

In [164]:
import pandas as pd
import yfinance as yf
import numpy as np

Step 2: acquire data （以亚马逊作为例子，spx作为市场基准）

Compute the Sharpe Ratio, the risk-adjusted returns and the Beta of every stock.

In [165]:
tickers = ["AMZN","^GSPC"]
mkt= yf.download(tickers, 
                 start = "2010-1-1",
                 progress= False)['Adj Close']
mkt.head()

,AMZN,^GSPC
Date,,
2010-01-04,6.6950,1132.989990
2010-01-05,6.7345,1136.520020
2010-01-06,6.6125,1137.140015
2010-01-07,6.5000,1141.689941
2010-01-08,6.6760,1144.979980


Step 3: log return

In [172]:
logrtn = np.log(1+mkt.pct_change().dropna())  
logrtn.head()

,AMZN,^GSPC
Date,,
2010-01-05,0.005883,0.003111
2010-01-06,-0.018282,0.000545
2010-01-07,-0.017160,0.003993
2010-01-08,0.026717,0.002878
2010-01-11,-0.024335,0.001745


Step 4: Compute covariance and market variance. use 252 as annuliza factor

In [173]:
annfactor = 252
covmatrix = logrtn.cov()*annfactor
covmatrix

,AMZN,^GSPC
AMZN,0.109139,0.034673
^GSPC,0.034673,0.031016


In [174]:
cov_with_market = covmatrix.iloc[0,1]
cov_with_market

0.03467326251397705

In [169]:
mktvar = logrtn.iloc[:,-1].var()*annfactor #Annualized
mktvar

0.03101608674971406

Step 5: Calculate Beta

In [170]:
beta = cov_with_market / mktvar
beta

1.1179122238654657

Step 6: Expected Return CAPM
Assumption: 
    A 10 year US government bond is a good proxy for a risk-free asset, with a yield of 2.5%

In [171]:
rf = 0.025
rm = logrtn['^GSPC'].mean() * 252
rp = (rm-rf)

amazon_capm_return = rf + beta * rp
amazon_capm_return

0.10902594187587064

Step 7: Sharpe Ratio

In [176]:
stdev_rtn = (logrtn['AMZN'].std()*250**0.5)
Sharpe = (amazon_capm_return - rf)/stdev_rtn
Sharpe

0.2553602485786315

Step 8: Create each part as functions

Import data:

In [177]:
from datetime import datetime

In [178]:
def import_stock_data(tickers, start = '2010-1-1', end = datetime.today().strftime('%Y-%m-%d')):
    data = pd.DataFrame()
    if len([tickers]) ==1:
        data[tickers] = yf.download(tickers, start = start)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(t, start = start)['Adj Close']
    return(data)

In [179]:
data = import_stock_data(["AMZN","^GSPC"])
data.head()

[*********************100%%**********************]  2 of 2 completed


,AMZN,^GSPC
Date,,
2010-01-04,6.6950,1132.989990
2010-01-05,6.7345,1136.520020
2010-01-06,6.6125,1137.140015
2010-01-07,6.5000,1141.689941
2010-01-08,6.6760,1144.979980


In [180]:
def calcbeta(data, stock, market,annfactor = 252):
    logrtn = np.log(1+data.pct_change().dropna())
    covmatrix = logrtn.cov()*250
    cov_w_market = covmatrix.loc[stock,market]
    market_var = logrtn[market].var()*250
    beta = cov_w_market / market_var
    return(beta)

In [181]:
beta = calcbeta(data,"AMZN","^GSPC")
beta

1.1179122238654657

In [182]:
def calccapm(data, stock, market, rf = 0.025,annfactor = 252):
    logrtn = np.log(1+data.pct_change().dropna())
    riskpremium = (logrtn[market].mean()*annfactor) - rf
    beta = calcbeta(data, stock, market)
    capmreturn = rf +beta * riskpremium
    return (capmreturn)

In [183]:
calccapm(data,"AMZN","^GSPC")

0.10902594187587064

In [184]:
def calcsharpe(data, stock, market, rf = 0.025,annfactor = 252):
    logrtn = np.log(1+data.pct_change().dropna())
    estrtn = calccapm(data, stock, market)
    stdev_rtn = (logrtn[stock].std()*annfactor**0.5)
    sharpe = (estrtn - rf)/stdev_rtn
    return (estrtn)

In [185]:
calcsharpe(data,"AMZN","^GSPC")

0.10902594187587064

In [186]:
def stock_capm(stktck,mkttck = "^GSPC", start = '2010-1-1', end = datetime.today().strftime('%Y-%m-%d'),rf = 0.025,annfactor = 252):
    data = import_stock_data([stktck,mkttck])
    beta = calcbeta(data,stktck,mkttck)
    capm = calccapm(data,stktck,mkttck)
    sharpe = calcsharpe(data,stktck,mkttck)
    capmdata = pd.DataFrame([beta,capm,sharpe], columns=[stktck], index=['Beta','Return','Sharpe'])
    return capmdata.T

In [187]:
stock_capm("AMZN")

[*********************100%%**********************]  2 of 2 completed


,Beta,Return,Sharpe
AMZN,1.117912,0.109026,0.109026
